<a href="https://colab.research.google.com/github/SohaAshraff/Football-Match-Analysis-System/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries
* Install all necessary packages for computer vision, OCR, and object tracking.

In [ ]:
!pip install inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install supervision git+https://github.com/roboflow/sports.git easyocr

  Cloning https://github.com/roboflow/sports.git to /tmp/pip-req-build-780noyae
  Running command git clone --filter=blob:none --quiet https://github.com/roboflow/sports.git /tmp/pip-req-build-780noyae
  Resolved https://github.com/roboflow/sports.git to commit 42c80c06b6b65a7f89455b89fe31cdf4c38ba227
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.5 MB/s eta 0:00:00
  Created wheel for sports: filename=sports-0.1.0-py3-none-any.whl size=10881 sha256=fb06cd3002c6693dc906442045f46495e5a42b4d386afdc4050989e2888e0b5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qrhm3z4p/wheels/6f/53/b4/9ae665e952091404183bcf40f7662061b

# Configuration & API Key Setup
**IMPORTANT: Enter your Roboflow API key into Secrets section in collab to access the player detection model.**
* 1. Click the key icon (🔑) in the left sidebar
* 2. Add a new secret with name: ROBOFLOW_API_KEY
* 3. Paste your API key as the value

# Import Libraries

In [ ]:
import supervision as sv
import cv2
import numpy as np
import easyocr
from tqdm import tqdm
from collections import defaultdict
from inference import get_model
from google.colab import userdata


print("All libraries imported successfully")

# Upload your video to collab and chang the path if it's has a different name


In [ ]:
SOURCE_VIDEO_PATH = "/content/CV_Task-1.mkv"
TARGET_VIDEO_PATH = "/content/final.mp4"

# Initialize Models
Load the Roboflow detection model and EasyOCR for jersey number recognition.

In [ ]:
RoKey = userdata.get('ROBOFLOW_API_KEY')

# Initialize Roboflow model
modelid = "football-players-detection-3zvbc/19"
PLAYER_DETECTION_MODEL = get_model(model_id=modelid, api_key=RoKey)

ocr_reader = easyocr.Reader(['en'], gpu=True)

# Detection class IDs
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3

STRIDE = 30

print("Models initialized successfully")

# Configuration Parameters
Set parameters for jersey number detection and tracking persistence.

In [ ]:
MIN_DETECTIONS_TO_LOCK = 3        # Need 3 consistent detections to lock a number
CONFIDENCE_LOCK_THRESHOLD = 0.5   # High confidence can lock with fewer detections
FRAMES_TO_KEEP_INVISIBLE = 150    # Keep number for 150 frames
MIN_CONFIDENCE_TO_ACCEPT = 0.2    # Minimum confidence for initial detection
CONFIDENCE_DECAY_RATE = 0.98      # Slower decay

print(" Parameters configured")
print(f"   Min detections to lock: {MIN_DETECTIONS_TO_LOCK}")
print(f"   Frames to keep invisible: {FRAMES_TO_KEEP_INVISIBLE}")

# OCR Preprocessing Functions
Define advanced image preprocessing techniques for better jersey number detection.


In [ ]:
def preprocess_crop_multi(crop):
    h, w = crop.shape[:2]
    upper_crop = crop[int(h * 0.05):int(h * 0.65), :]

    preprocessed_images = []

    # Method 1: Standard grayscale + CLAHE
    gray = cv2.cvtColor(upper_crop, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    preprocessed_images.append(enhanced)

    # Method 2: Adaptive threshold
    adaptive = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    preprocessed_images.append(adaptive)

    # Method 3: Otsu threshold with denoising
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    _, otsu = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    preprocessed_images.append(otsu)

    # Method 4: Inverted binary
    _, binary_inv = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    preprocessed_images.append(binary_inv)

    return preprocessed_images

print(" Preprocessing functions defined")

# Jersey Number Detection Function
Enhanced OCR function that tries multiple preprocessing methods.


In [ ]:
def detect_jersey_number_enhanced(crop, ocr_reader, return_conf=False):
    """Enhanced jersey number detection with multiple preprocessing methods and validation checks"""
    try:
        preprocessed_images = preprocess_crop_multi(crop)

        all_detections = []

        # Try OCR on all preprocessed versions
        for img in preprocessed_images:
            h, w = img.shape[:2]
            if h < 50:
                scale = 50 / h
                img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)

            results = ocr_reader.readtext(
                img,
                allowlist='0123456789',
                paragraph=False,
                min_size=10
            )

            for (bbox, text, conf) in results:
                # Validate: must be 1-2 digits, confidence > threshold
                if text.isdigit() and len(text) <= 2 and conf > 0.15:
                    all_detections.append((text, conf))

        if not all_detections:
            return ("", 0.0) if return_conf else ""

        number_scores = {}
        for num, conf in all_detections:
            if num not in number_scores:
                number_scores[num] = []
            number_scores[num].append(conf)

        # Calculate weighted score for each number
        best_number = ""
        best_score = 0.0

        for num, confs in number_scores.items():
            avg_conf = np.mean(confs)
            max_conf = max(confs)
            # Boost score if detected multiple times
            score = (avg_conf * 0.7 + max_conf * 0.3) * (1 + 0.15 * len(confs))

            if score > best_score:
                best_score = score
                best_number = num

        # Normalize score back to 0-1 range
        final_conf = min(best_score, 1.0)

        return (best_number, final_conf) if return_conf else best_number

    except Exception as e:
        return ("", 0.0) if return_conf else ""

print(" Jersey number detection function defined")

# Jersey Tracker Class
Track jersey numbers with confidence-based locking and temporal persistence.


In [ ]:
class JerseyTracker:
    """ Tracks jersey numbers with confidence-based locking and validation """
    def __init__(self):
        self.cache = {}

    class JerseyInfo:
        def __init__(self, number, confidence, frame):
            self.number = number
            self.confidence = confidence
            self.last_seen_frame = frame
            self.first_detected_frame = frame
            self.detection_count = 1
            self.detection_history = [number]
            self.is_locked = False

        def update(self, new_number, new_conf, frame):
            """Update jersey info with new detection"""
            self.last_seen_frame = frame

            if new_number:
                self.detection_history.append(new_number)
                # Keep last 10 detections for consistency check
                if len(self.detection_history) > 10:
                    self.detection_history.pop(0)

                # Check consistency
                most_common = max(set(self.detection_history),
                                 key=self.detection_history.count)

                if new_number == most_common:
                    # Reinforce existing number
                    self.confidence = min(self.confidence * 1.1 + new_conf * 0.2, 1.0)
                    self.detection_count += 1

                    # Lock if we have enough consistent detections
                    if (self.detection_count >= MIN_DETECTIONS_TO_LOCK or
                        self.confidence > CONFIDENCE_LOCK_THRESHOLD):
                        self.is_locked = True
                        self.number = new_number

                elif new_conf > self.confidence * 1.5:
                    # Only change if significantly more confident
                    if not self.is_locked:
                        self.number = new_number
                        self.confidence = new_conf
                        self.detection_count = 1

        def get_confidence_with_decay(self, current_frame):
            """Calculate current confidence with time-based decay"""
            frames_since_seen = current_frame - self.last_seen_frame

            if self.is_locked:
                decay = CONFIDENCE_DECAY_RATE ** (frames_since_seen / 60)
            else:
                decay = CONFIDENCE_DECAY_RATE ** (frames_since_seen / 30)

            return self.confidence * decay

        def should_display(self, current_frame):
            """Determine if number should be displayed"""
            frames_since_seen = current_frame - self.last_seen_frame
            current_conf = self.get_confidence_with_decay(current_frame)

            # Keep locked numbers much longer
            if self.is_locked:
                max_frames = FRAMES_TO_KEEP_INVISIBLE * 2
                min_conf = 0.1
            else:
                max_frames = FRAMES_TO_KEEP_INVISIBLE
                min_conf = 0.15

            return (frames_since_seen < max_frames and current_conf > min_conf)

print(" Jersey tracker class defined")

# Get Jersey Numbers Function
Main function to extract and track jersey numbers across frames.


In [ ]:
def get_jersey_numbers_persistent(crops, ocr_reader, tracker_ids, jersey_tracker, frame_count):
    """ Get jersey numbers with persistent tracking and confidence validation """
    jersey_numbers = []

    for i, crop in enumerate(crops):
        tracker_id = tracker_ids[i]

        # Detect number from current crop
        new_number, new_conf = detect_jersey_number_enhanced(
            crop, ocr_reader, return_conf=True
        )

        h, w = crop.shape[:2]
        size_factor = min((h * w) / 10000, 1.2)
        adjusted_conf = new_conf * size_factor

        # Update or create tracker
        if tracker_id in jersey_tracker.cache:
            info = jersey_tracker.cache[tracker_id]
            info.update(new_number, adjusted_conf, frame_count)
        else:
            if new_number and adjusted_conf >= MIN_CONFIDENCE_TO_ACCEPT:
                jersey_tracker.cache[tracker_id] = jersey_tracker.JerseyInfo(
                    new_number, adjusted_conf, frame_count
                )

        # Determine what to display
        if tracker_id in jersey_tracker.cache:
            info = jersey_tracker.cache[tracker_id]
            if info.should_display(frame_count):
                jersey_numbers.append(info.number)
            else:
                jersey_numbers.append("?")
        else:
            jersey_numbers.append("?")

    return jersey_numbers

print(" Jersey number tracking function defined")

# Team Classification Helper
Function to assign goalkeepers to the nearest team based on position.


In [ ]:
def resolve_goalkeepers_team_id(players, goalkeepers):
    if len(goalkeepers) == 0 or len(players) == 0:
        return np.array([])

    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)

    goalkeepers_team_id = []
    for goalkeeper_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        goalkeepers_team_id.append(0 if dist_0 < dist_1 else 1)

    return np.array(goalkeepers_team_id)

print(" Team classification helper defined")

# Legend Drawing Function

In [ ]:
def draw_legend(frame):
    """
    Draw a legend box in the top-left corner showing:
    - Team colors
    - Referee color
    - Ball indicator
    - Jersey number explanation
    """
    # Legend configuration
    padding = 15
    line_height = 35
    box_width = 330
    box_height = 300 # Increased height to fit all lines

    # Create semi-transparent overlay
    overlay = frame.copy()

    # Draw background (black with white border)
    cv2.rectangle(overlay,
                  (padding, padding),
                  (padding + box_width, padding + box_height),
                  (0, 0, 0), -1)
    cv2.rectangle(overlay,
                  (padding, padding),
                  (padding + box_width, padding + box_height),
                  (255, 255, 255), 2)

    # Blend overlay with frame
    alpha = 0.7
    result = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Starting position for text
    x = padding + 20
    y = padding + 35

    # Title
    cv2.putText(result, "LEGEND", (x, y),
                cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 2)
    y += line_height + 5

    # Team 1 - Blue
    cv2.ellipse(result, (x + 12, y - 8), (10, 6), 0, 0, 360, (255, 0, 0), -1)
    cv2.putText(result, "Team 1", (x + 35, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    y += line_height

    # Team 2 - Pink
    cv2.ellipse(result, (x + 12, y - 8), (10, 6), 0, 0, 360, (255, 0, 255), -1)
    cv2.putText(result, "Team 2", (x + 35, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
    y += line_height

    # Referee - Yellow
    cv2.ellipse(result, (x + 12, y - 8), (10, 6), 0, 0, 360, (0, 255, 255), -1)
    cv2.putText(result, "Referee", (x + 35, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    y += line_height

    # Ball - Triangle (Yellow)
    triangle_points = np.array([
        [x + 12, y - 15],
        [x + 2, y + 3],
        [x + 22, y + 3]
    ], dtype=np.int32)
    cv2.fillPoly(result, [triangle_points], (0, 255, 255))
    cv2.putText(result, "Ball", (x + 35, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    y += line_height



    # Jersey number explanation
    cv2.putText(result, "Jersey Numbers:", (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)
    y += line_height

    cv2.putText(result, "#  =  Player number", (x + 10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
    y += line_height

    cv2.putText(result, "?  =  Not detected", (x + 10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)

    return result

print(" Legend drawing function defined")

# Team Classification Training
Collect player crops and train the team classifier to distinguish between teams.


In [ ]:
from sports.common.team import TeamClassifier

# Collect crops for team classification
frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH, stride=STRIDE
)

crops = []
print("Collecting player crops for team classification...")
for frame in tqdm(frame_generator, desc='Collecting crops'):
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    detections = sv.Detections.from_inference(result)
    players_detections = detections[detections.class_id == PLAYER_ID]
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    crops += players_crops

print("Training team classifier...")
# Change device to 'cuda'
team_classifier = TeamClassifier(device="cuda")
team_classifier.fit(crops)

print(" Team classifier trained successfully!")
print(f" Total crops collected: {len(crops)}")

# Main Video Processing
Process the entire video with player detection, tracking, and jersey number recognition.

In [ ]:
# Initialize annotators
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER,
    text_thickness=2,
    text_scale=0.5
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

# Initialize tracker
tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=30
)
tracker.reset()

# Setup video processing
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info)
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

jersey_tracker = JerseyTracker()
frame_count = 0

print("Starting video processing with persistent jersey detection...")
print("Legend will be added to top-left corner of each frame")

with video_sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):
        frame_count += 1

        result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)

        # Ball detections
        ball_detections = detections[detections.class_id == BALL_ID]
        ball_detections.xyxy = sv.pad_boxes(ball_detections.xyxy, px=10)

        # Non-ball detections
        all_detections = detections[detections.class_id != BALL_ID]
        all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
        all_detections = tracker.update_with_detections(detections=all_detections)

        # Split detections
        goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
        players_detections = all_detections[all_detections.class_id == PLAYER_ID]
        referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

        # Predict teams
        players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
        if len(players_crops) > 0:
            players_detections.class_id = team_classifier.predict(players_crops)

        if len(goalkeepers_detections) > 0 and len(players_detections) > 0:
            goalkeepers_detections.class_id = resolve_goalkeepers_team_id(
                players_detections, goalkeepers_detections
            )

        referees_detections.class_id -= 1

        # Enhanced jersey number detection with persistence
        all_player_detections = sv.Detections.merge([players_detections, goalkeepers_detections])
        all_player_crops = [sv.crop_image(frame, xyxy) for xyxy in all_player_detections.xyxy]

        jersey_numbers = get_jersey_numbers_persistent(
            all_player_crops,
            ocr_reader,
            all_player_detections.tracker_id,
            jersey_tracker,
            frame_count
        )

        # Merge all detections
        all_detections = sv.Detections.merge(
            [players_detections, goalkeepers_detections, referees_detections]
        )
        all_detections.class_id = all_detections.class_id.astype(int)

        # Build labels
        labels = []
        jersey_idx = 0
        for class_id in all_detections.class_id:
            if class_id in [0, 1]:  # players and goalkeepers
                if jersey_idx < len(jersey_numbers):
                    labels.append(jersey_numbers[jersey_idx])
                else:
                    labels.append("?")
                jersey_idx += 1
            else:
                labels.append("")  # referees

        # Draw annotations
        annotated_frame = frame.copy()
        annotated_frame = ellipse_annotator.annotate(
            scene=annotated_frame,
            detections=all_detections
        )
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=all_detections,
            labels=labels
        )
        annotated_frame = triangle_annotator.annotate(
            scene=annotated_frame,
            detections=ball_detections
        )

        # Add legend to frame
        annotated_frame = draw_legend(annotated_frame)

        video_sink.write_frame(annotated_frame)

# Print statistics
locked_numbers = sum(1 for info in jersey_tracker.cache.values() if info.is_locked)
total_tracked = len(jersey_tracker.cache)

print("Processing complete! Output saved to:", TARGET_VIDEO_PATH)
print(f"Lock rate: {locked_numbers/total_tracked*100:.1f}%" if total_tracked > 0 else "N/A")

# Download Output Video

In [ ]:
from google.colab import files

# Download the output video
files.download(TARGET_VIDEO_PATH)

print("Video downloaded successfully!")